SIGNIFICANT OCCURRENCE REPORTED WHEN <N_1>, WMS STING, OVER FLEW <N_2>, HAWKER BEECHCRAFT BE36, AND LANDED RUNWAY 36 DENTON, TX 12/31 1527C.  CLOSEST PROXIMITY NOT REPORTED.  

AIRCRAFT OVERFLEW ANOTHER AIRCRAFT OR VEHICLE

NMAC REPORTED WHEN <C_a>, B748, AND <C_b>, B744, CAME WITHIN 300 FEET VERTICAL .36 MILE LATERAL SEPN AT 3,300 FEET 12 E LOS ANGELES, CA 12/29 1342P.  

NEAR COLLISION BETWEEN AIRCRAFT AND ANOTHER AIRCRAFT OR VEHICLE

<C_a>, PIPER PA31, DESCENDED 700 FEET BELOW ASSIGNED ALTITUDE OF 4,000 FEET 13 S FAIRBANKS, AK 12/17 1233L. NO CONFLICTS REPORTED.  

DESCENDED BELOW ASSIGNED/PUBLISHED ALTITUDE

<M_1>, CESSNA C208, FAILED TO MAINTAIN TWO WAY COMMUNICATION AT 8,000 FEET 28 NE BIMINI, BAHAMAS 12/29 1612L. NO CONFLICTS REPORTED. 

FAILED TO MAINTAIN COMMUNICATION

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, hamming_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
flightData = pd.read_csv('/content/drive/MyDrive/UCRIVERSIDE_20220617 (1).xlsx - EVENTS.csv')
flightData.head()

#df = flightData.copy(deep=True)

FileNotFoundError: ignored

In [ ]:
import re
narratives = flightData['NARRATIVE REDACTED']

# Remove all tags from narratives
narratives = [re.sub(r'<(.+?)>','',narrative) for narrative in narratives]
flightData['NARRATIVE REDACTED'] = narratives
flightData

In [ ]:
df2 = flightData.drop(flightData.iloc[:, 0:8], axis=1)
df2

In [ ]:
categories = list(df2.columns.values)

In [ ]:
# X = flightData['NARRATIVE REDACTED'] 
# Y = flightData.iloc[:, 8:]

# x_train, x_test, y_train, y_test = train_test_split(X, Y, 
#                                                     random_state=42, 
#                                                     test_size=0.2,
#                                                     shuffle=True)

# print('Training on {} examples with {} corresponding labels.'.format(x_train.shape, y_train.shape))
# print('Testing on', x_test.shape, 'examples with', y_test.shape, 'corresponding labels.')

In [ ]:
# x_train

In [ ]:
# y_test.to_numpy()

In [ ]:
def word_scorer(w1, w2 = None, with_similarity_score = False):
    '''
    Retrive the list of wordnet synonyms for a given word, and it's definition. Scores each against one specific synset.
    
    Args:
    w1 = word, text string, 'code'
    w2 = defined synset for similarity comparison, e.g. 'code.v.01' (default = None)
    with_similarity_score = set to True to include similarity scores of w1 with w2 synsets (default = False)
    
    Outputs:
    list of tuples (synonym name, synonym definition, similarity score)
    '''

    syns = []
    for i in range(len(wordnet.synsets(w1))):
        if with_similarity_score:
            if w2 is not None:
                syns.append((wordnet.synsets(w1)[i].name(),
                             wordnet.synsets(w1)[i].definition(),
                             wordnet.synset(w2).wup_similarity(wordnet.synsets(w1)[i])))
            else:
                print('with_similarity_score set to True, but no w2 defined')
                break
        else:
            syns.append((wordnet.synsets(w1)[i].name(),
                         wordnet.synsets(w1)[i].definition()))
    return syns

In [ ]:
def prep_phrase(phrase):
    '''
    Removes stopwords, punctuation from text, and converts into a list of word tokens
    
    Args:
    phrase = text string
    
    Outputs:
    list of word tokens
    '''
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(phrase)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return filtered_sentence

In [ ]:
def topic_scorer(phrase, topic, sim_thresh = 0.6, return_hits = False):
    '''
    For each word in a sentence, retrieves the synonym set. For each synonym we measure the wup_similarity
    to the topic at hand. If similarity > sim_threshold, the topic is said to have been mentioned.
    The wup_similarity threshold can be configured: where a higher threshold for increases the strictness of the word-to-topic similarity condition.
    If return_hits is set to True, the words in the phrase that were mapped to each topic will be returned.
    
    Args:
    filtered_sentence = tokenized sentence, preferrably stripped of stopwords
    topic = synset of the topic in question.
    sim_thresh = wup_similarity threshold for word and topic to be deemed similar enough (default 0.6)
    return_hits = return the words that matched to each topic (default = False)
    
    Outputs:
    Integer count of the number of mentions of the topic in the filtered_sentence
    '''
    
    phrase = prep_phrase(phrase)
    word_scores = []
    
    for w in range(len(phrase)):
        syns = wordnet.synsets(phrase[w])
        syns_sim = [topic.wup_similarity(syns[synonym]) for synonym in range(len(syns))]
        syns_sim = [sim if sim is not None else 0 for sim in syns_sim]
        try:
            syns_sim = np.max([1 if sim > sim_thresh else 0 for sim in syns_sim])
        except ValueError:
            syns_sim = 0
        word_scores.append(syns_sim)
    hits = [phrase[w] for w in range(len(phrase)) if word_scores[w] == 1]
        
    if return_hits:    
        return (np.sum(word_scores), hits)
    else:
        return np.sum(word_scores)

In [ ]:
def multi_topic_scorer(phrase, topic_dictionary, sim_thresh=0.6, return_hits=False):
    '''
    Takes a passage of text and maps words in that text to topics that have been defined in a topic dictionary.
    The wup_similarity threshold can be configured: where a higher threshold for increases the strictness of the word-to-topic similarity condition.
    If return_hits is set to True, the words in the phrase that were mapped to each topic will be returned.
    
    Args:
    phrase = passage of text
    topic_dictionary = dictionary where key:value is reader-friendly topic name:assigned synonym in wordnet
    sim_thresh = wup_similarity threshold for word and topic to be deemed similar enough (default 0.6)
    return_hits = return the words that matched to each topic (default = False)
    
    Outputs:
    sim_scores = dictionary where key:value is the reader-friendly topic name:number of synonyms present in the text
    '''

    sim_scores = {}
    
    for topic in list(topic_dictionary.keys()):
        
        for syn in topic_dictionary[topic]:
            topic_synset = wordnet.synset(syn)

            if topic in sim_scores.keys():
                sim_scores[topic] += topic_scorer(phrase, topic_synset, sim_thresh, return_hits)
            else:
                sim_scores[topic] = topic_scorer(phrase, topic_synset, sim_thresh, return_hits)
    
    return sim_scores

In [ ]:
# topic_dictionary = {
#     '1 AIRCRAFT DEPARTING, 1 AIRCRAFT CROSSING OR EXITING SAME RUNWAY': ['depart.v.03', 'depart.v.04', 'same.a.01', 'same.a.02'],
#     '1 AIRCRAFT LANDING, 1 AIRCRAFT CROSSING OR EXITING SAME RUNWAY': ['landing.n.03', 'same.a.01', 'same.a.02', 'cross.v.06'],
#     '1 AIRCRAFT LANDING, 1 AIRCRAFT DEPARTING INTERSECTING RUNWAYS': ['landing.n.03', 'depart.v.03', 'depart.v.04'],
#     '1 AIRCRAFT LANDING, 1 AIRCRAFT DEPARTING SAME RUNWAY': ['landing.n.03', 'depart.v.03', 'depart.v.04', 'same.a.01', 'same.a.02'],
#     '2 AIRCRAFT LANDING,  SAME RUNWAY': ['landing.n.03', 'same.a.01', 'same.a.02', 'both.s.01', 'final.s.02'],
#     'ABORTED TAKEOFF': ['abort.n.01', 'abort.v.01', 'stop.n.02', 'stop.v.01'],
#     'ACCEPTED DEPARTURE CLEARANCE MEANT FOR ANOTHER AIRCRAFT': ['depart.v.03', 'depart.v.04', 'accept.v.03', 'accept.v.04', 'another.s.01'],
#     'ACCEPTED LUAW MEANT FOR ANOTHER AIRCRAFT': ['accept.v.03', 'accept.v.04', 'intend.v.01', 'line_up.v.03', 'wait.n.02', 'wait.v.01', 'wait.v.02'],
#     'ACCEPTED/TOOK CLEARANCE/INSTRUCTION MEANT FOR ANOTHER AIRCRAFT': ['accept.v.03', 'accept.v.04', 'intend.v.01', 'take.v.09', 'take.v.27', 'clearance.n.03'],
#     'ACCEPTED/TOOK DEPARTURE CLEARANCE INTENDED FOR ANOTHER AIRCRAFT': ['accept.v.03', 'accept.v.04', 'intend.v.01', 'take.v.09', 'take.v.27', 'clearance.n.03', 'depart.v.03', 'depart.v.04', 'takeoff.n.01', 'takeoff.n.02'],
#     'ACCEPTED/TOOK DESCENT CLEARANCE INTENDED FOR ANOTHER AIRCRAFT': ['accept.v.03', 'accept.v.04', 'intend.v.01', 'take.v.09', 'take.v.27', 'clearance.n.03', 'descent.n.01', 'descent.n.03'],
#     'AIRCRAFT ATTEMPTED TO LAND ON CLOSED RUNWAY': ['low.r.01', 'low.a.02', 'closed.a.01', 'closed.s.09'],
#     'AIRCRAFT CLEARED FOR LOW APPROACH, PERFORMED TOUCH AND GO': ['touch.v.08', 'go.v.03'],
#     'AIRCRAFT CLEARED TO DEPART RUNWAY, ATTEMPTED TO DEPART TAXIWAY': ['depart.v.03', 'depart.v.04', 'cancel.v.01', 'stop.v.01', 'taxiway.n.01'],
#     'AIRCRAFT CLEARED TO LAND ON RUNWAY, LANDED ON TAXIWAY': ['land.v.02', 'taxiway.n.01'],
#     'AIRCRAFT CLEARED TO LAND, PERFORMED TOUCH AND GO': ['land.v.02', 'touch.v.08', 'go.v.03'],
#     'AIRCRAFT FIRE': ['fire.n.01', 'fire.n.03', 'burn.v.01'],
#     'AIRCRAFT MANEUVERED (AIRBORNE/SURFACE TO AVOID ANOTHER AIRCRAFT OR VEHICLE': ['avoid.v.01'],
#     'AIRCRAFT OVERFLEW ANOTHER AIRCRAFT OR VEHICLE': ['pass_over.v.04'],
    # 'ATC CANCELED DEPARTURE CLEARANCE',
    # 'ATC DID NOT ISSUE TRAFFIC ADVISORY',
    # 'ATC INSTRUCTED AIRCRAFT TO GO AROUND',
    # 'ATC INSTRUCTED AIRCRAFT/VEHICLE TO HOLD SHORT',
    # 'ATC ISSUED TRAFFIC ADVISORY',
    # 'ATC USED INCORRECT CALLSIGN',
    # 'ATTEMPTED TO DEPART WITHOUT CLEARANCE',
    # 'ATTEMPTED TO DEPART WRONG RUNWAY',
    # 'ATTEMPTED TO LAND ON WRONG RUNWAY',
    # 'CARELESS, RECKLESS, OR DANGEROUS ACTION',
    # 'CHANGED FREQUENCY WITHOUT AUTHORIZATION',
    # 'CLEARED TO DEPART RUNWAY, ATTEMPTED TO DEPART TAXIWAY',
    # 'CLEARED TO LAND, PERFORMED TOUCH AND GO',
    # 'CLIMBED ABOVE ASSIGNED/PUBLISHED ALTITUDE',
    # 'CLIMBED ABOVE CROSSING RESTRICTION ALTITUDE',
    # 'COLLISION BETWEEN AIRCRAFT AND ANOTHER AIRCRAFT OR VEHICLE',
    # 'COLLISION WITH TERRAIN, OBSTACLES',
    # 'CROP DUSTING, AERIAL APPLICATION',
    # 'CROSSED RUNWAY HOLD LINE WITHOUT CLEARANCE',
    # 'DEPARTED FROM CLOSED RUNWAY',
    # 'DEPARTED FROM RUNWAY WITHOUT CLEARANCE',
    # 'DEPARTED FROM TAXIWAY WITHOUT CLEARANCE',
    # 'DEPARTED RAMP WITHOUT AUTHORIZATION',
    # 'DEPARTED WITHOUT CLEARANCE/AUTHORIZATION',
    # 'DEPARTED WITHOUT IFR CLEARANCE',
    # 'DEPARTED WITHOUT IFR RELEASE',
    # 'DEPARTED WITHOUT RELEASE CLEARANCE',
    # 'DEPARTED WRONG RUNWAY',
    # 'DESCENDED BELOW ASSIGNED/PUBLISHED ALTITUDE',
    # 'DESCENDED BELOW CROSSING RESTRICTION ALTITUDE',
    # 'ENTERED AIRSPACE WITHOUT AUTHORIZATION',
    # 'ENTERED MOVEMENT AREA WITHOUT CLEARANCE',
    # 'ENTERED RUNWAY SAFETY AREA WITHOUT CLEARANCE',
    # 'ENTERED SFRA WITHOUT AUTHORIZATION',
    # 'ENTERED SPECIAL USE AIRSPACE WITHOUT AUTHORIZATION',
    # 'ENTERED/CROSSED CLOSED RUNWAY WITHOUT CLEARANCE',
    # 'ENTERED/CROSSED RUNWAY WITHOUT CLEARANCE',
    # 'ENTERED/CROSSED TAXIWAY WITHOUT CLEARANCE',
    # 'EXECUTED TOUCH AND GO WITHOUT CLEARANCE',
    # 'EXIT SPECIAL USE AIRSPACE WITHOUT CLEARANCE (SPILL OUT)',
    # 'FAILED TO CLEAR HOLD LINE',
    # 'FAILED TO CLEAR RUNWAY',
    # 'FAILED TO CLEAR RUNWAY SAFETY AREA',
    # 'FAILED TO CLIMB TO ASSIGNED/PUBLISHED ALTITUDE',
    # 'FAILED TO CLOSE/CANCEL IFR FLIGHT PLAN',
    # 'FAILED TO COMPLY WITH CROSSING RESTRICTION',
    # 'FAILED TO DESCEND TO MEET CROSSING RESTRICTION',
    # 'FAILED TO ESTABLISH COMMUNICATION',
    # 'FAILED TO EXIT/CLEAR RUNWAY',
    # 'FAILED TO FLY ASSIGNED AIRSPEED',
    # 'FAILED TO FLY ASSIGNED ALTITUDE',
    # 'FAILED TO FLY ASSIGNED APPROACH',
    # 'FAILED TO FLY ASSIGNED APPROACH COURSE',
    # 'FAILED TO FLY ASSIGNED ARRIVAL ROUTE',
    # 'FAILED TO FLY ASSIGNED/PUBLISHED APPROACH/ARRIVAL CLEARANCE/PROCEDURE',
    # 'FAILED TO FLY ASSIGNED/PUBLISHED ARRIVAL CLEARANCE/PROCEDURE',
    # 'FAILED TO FLY ASSIGNED/PUBLISHED MISSED APPROACH',
    # 'FAILED TO FLY/FOLLOW ASSIGNED COURSE/CLEARANCE/heading',
    # 'FAILED TO FOLLOW TAXI INSTRUCTIONS',
    # 'FAILED TO FOLLOW/FLY ASSIGNED/PUBLISHED DEPARTURE INSTRUCTIONS/CLEARANCE',
    # 'FAILED TO FOLLOW/FLY DEPARTURE PROCEDURE',
    # 'FAILED TO MAINTAIN COMMUNICATION',
    # 'FAILED TO MEET CROSSING RESTRICTION ALTITUDE',
    # 'FLIGHT BELOW OR DESCENDED BELOW MINIMUM VECTOR ALTITUDE (MVA)',
    # 'GO AROUND',
    # 'ILLNESS, INJURY',
    # 'LANDED  WITHOUT CLEARANCE',
    # 'LANDED CLOSED AIRPORT',
    # 'LANDED ON CLOSED RUNWAY',
    # 'LANDED ON TAXIWAY',
    # 'LANDED ON TAXIWAY WITHOUT CLEARANCE',
    # 'LANDED RUNWAY WITHOUT CLEARANCE',
    # 'LANDED WITHOUT CLEARANCE',
    # 'LANDED WRONG AIRPORT',
    # 'LANDED WRONG RUNWAY',
    # 'LASER',
    # 'LINED UP TO LAND ON TAXIWAY',
    # 'LINED UP TO LAND WRONG RUNWAY',
    # 'LOSS OF SEPARATION',
    # 'LOSS OF SEPARATION WITH TERRAIN, OBSTACLES',
    # 'LUAW': ['line_up.v.03', 'wait.n.02', 'wait.v.01', 'wait.v.02'],
    # 'LUAW WRONG RUNWAY': [],
    # 'MIDAIR COLLISION',
    # 'MILITARY OPERATIONS',
    # 'MISSED APPROACH',
    # 'NEAR COLLISION BETWEEN AIRCRAFT AND ANOTHER AIRCRAFT OR VEHICLE',
    # 'OVERFLEW AIRCRAFT ON RUNWAY',
    # 'PARACHUTE MISHAP',
    # 'PARACHUTE/SKY DIVING OPERATIONS',
    # 'PARALLEL RUNWAYS',
    # 'RUNWAY INCURSION',
    # 'SPACE VEHICLE, FACILITY?',
    # 'TAXI WITHOUT CLEARANCE',
    # 'TAXIED ON CLOSED TAXIWAY WITHOUT CLEARANCE',
    # 'TAXIED TO PARKING WITHOUT CLEARANCE',
    # 'TAXIED TO RUNWAY WITHOUT CLEARANCE',
    # 'TCAS ALERT / RA',
    # 'TCAS RA',
    # 'TOUCH AND GO WRONG RUNWAY',
    # 'TRAINING IN PROGRESS',
    # 'TURBULENCE',
    # 'UNMANNED AIRCRAFT SYSTEM',
    # 'VEHICLE ON RUNWAY WITHOUT AUTHORIZATION',
    # 'WAKE TURBULENCE',
    # 'WORKLOAD, DISTRACTION': []
# }

In [ ]:
# topic_dictionary = {
#     '1 AIRCRAFT DEPARTING, 1 AIRCRAFT CROSSING OR EXITING SAME RUNWAY': 'depart.v.03',
#     '1 AIRCRAFT LANDING, 1 AIRCRAFT CROSSING OR EXITING SAME RUNWAY': 'landing.n.03',
#     '1 AIRCRAFT LANDING, 1 AIRCRAFT DEPARTING INTERSECTING RUNWAYS': 'landing.n.03',
#     '1 AIRCRAFT LANDING, 1 AIRCRAFT DEPARTING SAME RUNWAY': 'same.a.01',
#     '2 AIRCRAFT LANDING,  SAME RUNWAY': 'both.s.01',
#     'ABORTED TAKEOFF': 'abort.n.01',
#     'ACCEPTED DEPARTURE CLEARANCE MEANT FOR ANOTHER AIRCRAFT': 'another.s.01',
#     'ACCEPTED LUAW MEANT FOR ANOTHER AIRCRAFT': 'line_up.v.03',
#     'ACCEPTED/TOOK CLEARANCE/INSTRUCTION MEANT FOR ANOTHER AIRCRAFT': 'clearance.n.03',
#     'ACCEPTED/TOOK DEPARTURE CLEARANCE INTENDED FOR ANOTHER AIRCRAFT': 'takeoff.n.02',
#     'ACCEPTED/TOOK DESCENT CLEARANCE INTENDED FOR ANOTHER AIRCRAFT': 'descent.n.03',
#     'AIRCRAFT ATTEMPTED TO LAND ON CLOSED RUNWAY': 'closed.s.09',
#     'AIRCRAFT CLEARED FOR LOW APPROACH, PERFORMED TOUCH AND GO': 'touch.v.08',
#     'AIRCRAFT CLEARED TO DEPART RUNWAY, ATTEMPTED TO DEPART TAXIWAY': 'taxiway.n.01',
#     'AIRCRAFT CLEARED TO LAND ON RUNWAY, LANDED ON TAXIWAY': 'land.v.02',
#     'AIRCRAFT CLEARED TO LAND, PERFORMED TOUCH AND GO': 'touch.v.08',
#     'AIRCRAFT FIRE': 'fire.n.01',
#     'AIRCRAFT MANEUVERED (AIRBORNE/SURFACE TO AVOID ANOTHER AIRCRAFT OR VEHICLE': 'avoid.v.01',
#     'AIRCRAFT OVERFLEW ANOTHER AIRCRAFT OR VEHICLE': 'pass_over.v.04',
#     'ILLNESS, INJURY': 'injury.n.02'
# }

In [ ]:
word_scorer('two')

In [ ]:
topic_dictionary = {
    'AIRCRAFT MANEUVERED (AIRBORNE/SURFACE TO AVOID ANOTHER AIRCRAFT OR VEHICLE': ['avoid.v.01'],
    'ATC CANCELED DEPARTURE CLEARANCE': ['cancel.v.01', 'clearance.n.03'],
    'ATC INSTRUCTED AIRCRAFT TO GO AROUND': ['around.r.07'],
    'CLIMBED ABOVE ASSIGNED/PUBLISHED ALTITUDE': ['climb.v.01', 'above.r.02', 'assigned.a.01', 'altitude.n.01'],
    'CROSSED RUNWAY HOLD LINE WITHOUT CLEARANCE': ['cross.v.05', 'line.n.11', 'clearance.n.03'],
    'DEPARTED WITHOUT CLEARANCE/AUTHORIZATION': ['depart.v.03', 'clearance.n.03'],
    'DESCENDED BELOW ASSIGNED/PUBLISHED ALTITUDE': ['descend.v.01', 'below.r.01', 'assigned.a.01', 'altitude.n.01'],
    'ENTERED AIRSPACE WITHOUT AUTHORIZATION': ['enter.v.01', 'airspace.n.01'],
    'ENTERED/CROSSED RUNWAY WITHOUT CLEARANCE': ['enter.v.01', 'runway.n.04', 'clearance.n.03'],
    'ENTERED/CROSSED TAXIWAY WITHOUT CLEARANCE': ['enter.v.01', 'taxiway.n.01', 'clearance.n.03'],
    'FAILED TO COMPLY WITH CROSSING RESTRICTION': ['fail.v.01', 'cross.v.05', 'restriction.n.03'],
    'FAILED TO FLY/FOLLOW ASSIGNED COURSE/CLEARANCE/heading': ['fail.v.01', 'assigned.a.01'],
    'FAILED TO FOLLOW TAXI INSTRUCTIONS': ['fail.v.01', 'taxi.v.01'],
    'FAILED TO FOLLOW/FLY DEPARTURE PROCEDURE': ['fail.v.01', 'depart.v.03'],
    'FAILED TO MAINTAIN COMMUNICATION': ['fail.v.01', 'two.s.01'],
    'ILLNESS, INJURY': ['fatal.a.01', 'dead.n.01', 'unresponsive.a.01', 'hospital.n.02', 'ill.a.01', 'injure.v.01', 'stroke.n.03', 'blur.v.02', 'pain.n.03', 'crash.v.03', 'cardiac.a.01', 'suffer.v.02'],
    'LANDED WITHOUT CLEARANCE': ['land.v.02', 'clearance.n.03'],
    'SPACE VEHICLE, FACILITY?': ['national_aeronautics_and_space_administration.n.01', 'shuttle.n.02'],
    'UNIDENTIFIED': ['unidentified.s.01'],
    'VEHICLE ON RUNWAY WITHOUT AUTHORIZATION': ['vehicle.n.01', 'authorization.n.04']
}

In [ ]:
#tempsubset = flightData.loc[(flightData.loc[:, categories[0:18]] != 0).any(axis=1)]
#tempsubset

In [ ]:
#for index, row in flightData.iloc[2817:2818, :].iterrows():
print(multi_topic_scorer(flight['NARRATIVE REDACTED'][5], topic_dictionary, sim_thresh=0.7, return_hits=True))